In [60]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='ignore') 

In [88]:
train_data1 = pd.read_csv('C:/Users/user/Desktop/데이콘/open/train.csv')
train_data2 = pd.read_csv('C:/Users/user/Desktop/데이콘/open/international_trade.csv')
test_data = pd.read_csv('C:/Users/user/Desktop/데이콘/open/test.csv')

In [89]:
print(train_data1['item'].value_counts())
print(test_data['item'].value_counts())

item
TG    15230
BC    13707
RD    12184
CR    10661
CB     7615
Name: count, dtype: int64
item
TG    280
BC    252
RD    224
CR    196
CB    140
Name: count, dtype: int64


In [91]:
train_data2

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38
...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71
1270,2023-02,사과,135165,351,0,0,351
1271,2023-02,배,2206012,5411,1,0,5411
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0


In [92]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_data1['year'] = train_data1['timestamp'].apply(lambda x : int(x[0:4]))
train_data1['month'] = train_data1['timestamp'].apply(lambda x : int(x[5:7]))
train_data1['day'] = train_data1['timestamp'].apply(lambda x : int(x[8:10]))

test_data['year'] = test_data['timestamp'].apply(lambda x : int(x[0:4]))
test_data['month'] = test_data['timestamp'].apply(lambda x : int(x[5:7]))
test_data['day'] = test_data['timestamp'].apply(lambda x : int(x[8:10]))

train_data2['year'] = train_data2['기간'].apply(lambda x : int(x[0:4]))
train_data2['month'] = train_data2['기간'].apply(lambda x : int(x[5:7]))

In [93]:
train_data2.loc[train_data2['품목명']=='꽃양배추와 브로콜리(broccoli)','품목명'] = '브로콜리'
train_data2.loc[train_data2['품목명']=='순무','품목명'] = '무'

In [94]:
fruits_dict = {'감귤':'TG' ,'브로콜리':'BC' ,'무':'RD' ,'당근':'CR' ,'양배추':'CB'}
fruits_dict

{'감귤': 'TG', '브로콜리': 'BC', '무': 'RD', '당근': 'CR', '양배추': 'CB'}

In [95]:
train_data2['품목명'] = train_data2['품목명'].map(fruits_dict)

In [96]:
train_data2

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,2019-01,NaN,356571,990,0,0,990,2019,1
1,2019-01,NaN,821330,222,4003206,1118,-896,2019,1
2,2019-01,NaN,60,1,93405,128,-127,2019,1
3,2019-01,BC,160,1,638913,563,-562,2019,1
4,2019-01,NaN,0,0,7580,38,-38,2019,1
...,...,...,...,...,...,...,...,...,...
1269,2023-02,NaN,0,0,23830,71,-71,2023,2
1270,2023-02,NaN,135165,351,0,0,351,2023,2
1271,2023-02,NaN,2206012,5411,1,0,5411,2023,2
1272,2023-02,NaN,5,0,0,0,0,2023,2


In [97]:
train_data2 = train_data2.dropna()

In [98]:
train_data2['가격/kg'] = ((train_data2['수출 금액'] + train_data2['수입 금액']) * 1000 * 1300) / (train_data2['수출 중량'] + train_data2['수입 중량'])  

In [99]:
td_TG = train_data2[train_data2['품목명'] == 'TG']
td_BC = train_data2[train_data2['품목명'] == 'BC']
td_RD = train_data2[train_data2['품목명'] == 'RD']
td_CR = train_data2[train_data2['품목명'] == 'CR']
td_CB = train_data2[train_data2['품목명'] == 'CB']

In [100]:
td_CB

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month,가격/kg
5,2019-01,CB,184650,94,395802,90,4,2019,1,412.092645
30,2019-02,CB,182636,69,336142,77,-8,2019,2,365.859770
53,2019-03,CB,854925,354,356514,94,260,2019,3,480.750578
77,2019-04,CB,6494616,2658,343463,105,2553,2019,4,525.279103
101,2019-05,CB,1461639,662,270972,95,567,2019,5,567.986698
128,2019-06,CB,3286366,1656,271444,69,1586,2019,6,630.303473
155,2019-07,CB,1676629,798,472041,119,679,2019,7,554.808323
182,2019-08,CB,2465625,1154,280140,73,1081,2019,8,580.930997
209,2019-09,CB,1187087,566,226329,60,507,2019,9,575.768210
237,2019-10,CB,201297,105,635187,151,-46,2019,10,397.855787


In [86]:
train_data1[(train_data1['item'] == 'CB') & (train_data1['month'] == 1) & (train_data1['supply(kg)'] != 0)]

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day
25895,CB_A_J_20190105,2019-01-05,CB,A,J,4112.0,374.0,2019,1,5
25898,CB_A_J_20190108,2019-01-08,CB,A,J,4480.0,250.0,2019,1,8
25899,CB_A_J_20190109,2019-01-09,CB,A,J,3760.0,273.0,2019,1,9
25909,CB_A_J_20190119,2019-01-19,CB,A,J,3760.0,321.0,2019,1,19
25914,CB_A_J_20190124,2019-01-24,CB,A,J,7680.0,548.0,2019,1,24
...,...,...,...,...,...,...,...,...,...,...
57837,CB_F_J_20230126,2023-01-26,CB,F,J,8568.0,973.0,2023,1,26
57838,CB_F_J_20230127,2023-01-27,CB,F,J,76920.0,709.0,2023,1,27
57839,CB_F_J_20230128,2023-01-28,CB,F,J,81728.0,835.0,2023,1,28
57841,CB_F_J_20230130,2023-01-30,CB,F,J,156840.0,873.0,2023,1,30


In [87]:
train_data1[(train_data1['item'] == 'CB') & (train_data1['month'] == 1) & (train_data1['supply(kg)'] == 0)]

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day
25891,CB_A_J_20190101,2019-01-01,CB,A,J,0.0,0.0,2019,1,1
25892,CB_A_J_20190102,2019-01-02,CB,A,J,0.0,0.0,2019,1,2
25893,CB_A_J_20190103,2019-01-03,CB,A,J,0.0,0.0,2019,1,3
25894,CB_A_J_20190104,2019-01-04,CB,A,J,0.0,0.0,2019,1,4
25896,CB_A_J_20190106,2019-01-06,CB,A,J,0.0,0.0,2019,1,6
...,...,...,...,...,...,...,...,...,...,...
57826,CB_F_J_20230115,2023-01-15,CB,F,J,0.0,0.0,2023,1,15
57833,CB_F_J_20230122,2023-01-22,CB,F,J,0.0,0.0,2023,1,22
57834,CB_F_J_20230123,2023-01-23,CB,F,J,0.0,0.0,2023,1,23
57835,CB_F_J_20230124,2023-01-24,CB,F,J,0.0,0.0,2023,1,24
